In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./data/train.csv")

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
y = df.Survived

In [6]:
# filter data
df = df[["Age","Embarked","Sex"]]

In [7]:
df.isna().sum()

Age         177
Embarked      2
Sex           0
dtype: int64

In [8]:
df["Embarked"].fillna(inplace=True,value="S")
df.Age.fillna(inplace=True,axis=0,value=int(df.Age.mean()))

In [9]:
df

,Age,Embarked,Sex
0,22.0,S,male
1,38.0,C,female
2,26.0,S,female
3,35.0,S,female
4,35.0,S,male
...,...,...,...
886,27.0,S,male
887,19.0,S,female
888,29.0,S,female
889,26.0,C,male


In [10]:
df.isna().sum()

Age         0
Embarked    0
Sex         0
dtype: int64

In [11]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [12]:
ohe = OneHotEncoder()
features = ["Embarked","Sex"]
transformer = ColumnTransformer(
    [["ohe",
      ohe,
      features
     ]],
    remainder="passthrough"
)
# trans = transformer.fit_transform(df)
# transdf = pd.DataFrame(trans)
transdf = pd.get_dummies(df)

In [13]:
transdf

,Age,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male
0,22.0,0,0,1,0,1
1,38.0,1,0,0,1,0
2,26.0,0,0,1,1,0
3,35.0,0,0,1,1,0
4,35.0,0,0,1,0,1
...,...,...,...,...,...,...
886,27.0,0,0,1,0,1
887,19.0,0,0,1,1,0
888,29.0,0,0,1,1,0
889,26.0,1,0,0,0,1


In [14]:
from sklearn.linear_model import LogisticRegression

In [15]:
from sklearn.model_selection import train_test_split
import numpy as np
np.random.seed(70)

In [16]:
xTrain, xTest, yTrain, yTest = train_test_split(transdf,y,test_size=0.2)

In [17]:
clf = LogisticRegression()

In [18]:
clf.fit(xTrain,yTrain)
clf.score(xTest,yTest)

0.7988826815642458

In [19]:
import pickle

In [20]:
with open("./trainedModel/trained.pkl","wb") as file:
    pickle.dump(clf,file)

In [21]:
xTest

,Age,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male
215,31.0,1,0,0,1,0
513,54.0,1,0,0,1,0
53,29.0,0,0,1,1,0
221,27.0,0,0,1,0,1
883,28.0,0,0,1,0,1
...,...,...,...,...,...,...
371,18.0,0,0,1,0,1
551,27.0,0,0,1,0,1
177,50.0,1,0,0,1,0
7,2.0,0,0,1,0,1


In [22]:
yTest

215    1
513    1
53     1
221    0
883    0
      ..
371    0
551    0
177    0
7      0
161    1
Name: Survived, Length: 179, dtype: int64

In [23]:
test = np.array([[31,"C","female"]])
test

array([['31', 'C', 'female']], dtype='<U11')

In [24]:
testDf = pd.DataFrame(test,columns=["Age","Embarked","Sex"])
testDf["Age"] = testDf.Age.astype(int)
testDf

,Age,Embarked,Sex
0,31,C,female


In [25]:
test = pd.DataFrame([],columns=xTrain.columns).append(pd.get_dummies(testDf)).fillna(0)

In [26]:
clf.predict(test)

array([1], dtype=int64)